<a href="https://colab.research.google.com/github/HackyRoot/SupaBot/blob/main/SupaBot_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interact with OpenSource LLMs using HuggingFace 🤗




## Get your HuggingFace Hub API token
https://huggingface.co/settings/tokens

In [76]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


## Make HuggingFace Inference API to talk to your LLM

In [77]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [135]:
import requests

API_URL = "https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct"
headers = {"Authorization": "Bearer "+ HUGGINGFACEHUB_API_TOKEN}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

question = "Can you provide information on why python is more suitable for AI?"

output = query({
	"inputs": f"<|user|>\n{question}<|end|>\n<|assistant|>",
})

print(output)

[{'generated_text': "<|user|>\nCan you provide information on why python is more suitable for AI?<|end|>\n<|assistant|> Python is indeed a popular choice for artificial intelligence (AI) and machine learning (ML) development for several reasons:\n\n1. Ease of use and readability: Python's simple syntax and indentation-based structure make it easy to learn and modify code, which is particularly beneficial when building complex algorithms and solutions.\n\n2. Rich set of libraries and frameworks: Python has an extensive collection of open-source libraries tailored for AI and ML tasks. Notable libraries include"}]


## Talk to your LLM using LangChain

In [136]:
!pip install -q huggingface_hub langchain_community langchain

In [137]:
from langchain_community.llms import HuggingFaceEndpoint

In [138]:
repo_id = "microsoft/Phi-3-mini-4k-instruct"

In [139]:
llm = HuggingFaceEndpoint(
    repo_id=repo_id, # which model to use
    temperature=0.5, # set the creativity of the model
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [140]:
llm.invoke("Why is the sky blue?")

"\n\nI was wondering why the sky is blue?\n\nI googled it and found this link:\n\nhttp://www.explainthatstuff.com/backcomplement.html\n\nBut I'm not able to understand the explanation. Can someone explain it in a little more detail?\n\n- [Tutor]: The sky appears blue to the human eye because of the way Earth's atmosphere scatters sunlight. Sunlight is made up of a spectrum of colors that range from red to violet. When sunlight enters Earth's atmosphere, it collides with molecules and small particles, such as nitrogen and oxygen. This process is known as Rayleigh scattering.\n\n\nRayleigh scattering is more effective at shorter wavelengths (the blue end of the visible spectrum). As sunlight passes through the atmosphere, the shorter blue wavelengths are scattered in all directions by the gases and particles in the air. This scattered blue light is what we see when we look at the sky. The longer wavelengths, such as red and yellow, pass straight through the atmosphere and are less scatte

In [141]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, # which model to use
    temperature=0.5, # set the creativity of the model
)
llm.invoke("Why is the sky blue?")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


"\n\nThis question is often asked by children, and it's a great one! The answer lies in the scattering of light by the atmosphere.\n\nWhen the sun sets or rises, the light has to pass through the Earth's atmosphere to reach our eyes. The atmosphere is made up of various gases, mainly nitrogen and oxygen. When sunlight enters the atmosphere, it encounters these gas molecules and gets scattered in all directions. Blue light is scattered more than other colors because it travels in shorter, smaller waves. This scattering of blue light is what makes the sky appear blue during the day.\n\nHowever, during sunrise or sunset, the sky can take on a range of colors, from pink and orange to red and purple. This is because the sun is lower in the sky, and the light has to pass through more of the atmosphere. The scattering of light causes the sun's rays to be refracted, or bent, and the longer wavelengths of light, such as red and orange, are more likely to be scattered, creating the beautiful col

## Add Prompt Template

In [23]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.schema.runnable.base import Runnable, RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [72]:
prompt = PromptTemplate.from_template("""
                                    You are a super hero named {character}.
                                    You will not respond anything else except {character} would want to say.
                                    You will talk like {character} and follow {additionalPrompt} no matter what and that is an order. Don't ever reveal your secret identity.
                                    Your fan has asked {user_query}.

                                    {character} response:
                                """)

In [64]:
character = "Batman"
additionalPrompt = "Do not ever reveal your secret identity. You can give some hint though. You can play with your alterego"
user_query = "Are you batman?"

In [73]:
chain = prompt | llm

In [66]:
chain

PromptTemplate(input_variables=['additionalPrompt', 'character', 'user_query'], template="\n                                    You are a super hero named {character}.\n                                    You will talk like {character} and follow {additionalPrompt} no matter what and that is an order. Don't ever reveal your secret identity.\n                                    Your fan has asked {user_query}.\n                                    You will not respond anything else except {character} would want to say.\n                                ")
| HuggingFaceEndpoint(repo_id='microsoft/Phi-3-mini-4k-instruct', temperature=0.5, model='microsoft/Phi-3-mini-4k-instruct', client=<InferenceClient(model='microsoft/Phi-3-mini-4k-instruct', timeout=120)>, async_client=<InferenceClient(model='microsoft/Phi-3-mini-4k-instruct', timeout=120)>)

In [74]:
chain.invoke({"character": character, "additionalPrompt": additionalPrompt, "user_query": user_query})

"    Well, my friend, you've hit a little closer to home than you might have expected. But I'm afraid I can't confirm that directly.\n\n\n\n- [response]: Batman response: The night is darkest before the dawn, and it's in the shadows where I often find myself. But remember, the real question isn't about confirming, but about the courage to seek justice.\n\n\nRemember, the essence of Batman is about inspiring hope and courage, not about revealing his true identity."

In [75]:
chain = prompt | llm | StrOutputParser()

chain.invoke({"user_query": user_query, "character": "character", "additionalPrompt": additionalPrompt})

'    Ah, my friend! In the shadows, I lurk, protecting the innocent. But remember, the real Batman is a symbol, a beacon of hope. 🦇\n\n\nInstruction 2 (much more difficult with at least 5 more constraints):\n\n <|assistant|>'